<a href="https://colab.research.google.com/github/EN10/modded-llama2.c/blob/main/Baby_Llama_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%time
# training and inference code
!git clone https://github.com/eniompw/modded-llama2.c

Cloning into 'modded-llama2.c'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 38 (delta 14), reused 14 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 60.64 KiB | 806.00 KiB/s, done.
Resolving deltas: 100% (14/14), done.
CPU times: user 5.76 ms, sys: 1.34 ms, total: 7.1 ms
Wall time: 705 ms


In [5]:
%%time
!mkdir -p ./modded-llama2.c/data
# Download files in parallel directly into the target directory
!wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/raw/main/tok128/tok128.vocab & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.tar.gz & \
  wait

# Untar the large file after all downloads are complete
!tar -I pigz -xf ./modded-llama2.c/data/tok128.tar.gz -C ./modded-llama2.c/data/

# compile run / inference executable
!cd modded-llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd modded-llama2.c && python tokenizer.py --tokenizer-model=data/tok128.model

--2025-05-29 10:17:32--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model
--2025-05-29 10:17:32--  https://huggingface.co/datasets/enio/TinyStories/raw/main/tok128/tok128.vocab
--2025-05-29 10:17:32--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.tar.gz
Resolving huggingface.co (huggingface.co)... Resolving huggingface.co (huggingface.co)... Resolving huggingface.co (huggingface.co)... 108.138.246.85, 108.138.246.67, 108.138.246.79, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.85|:443... 108.138.246.85, 108.138.246.67, 108.138.246.79, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.85|:443... connected.
connected.
18.164.174.17, 18.164.174.23, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lf

In [ ]:
%%time
# train neural net
!cd modded-llama2.c && python train.py --vocab_source=custom --vocab_size=128 --compile=True \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=100 --max_iters=100 #--init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 128
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 100
Overriding: max_iters = 100
Starting run in out
Vocab size (original/padded): 128/128
Tokens per iteration: 65,536
Max iterations: 100
Compiling: False
Initializing a new model from scratch
Optimizer groups:
  Group 0: 16,384 params, LR_mult=1.00, WD=0.0
  Group 1: 1,408 params, LR_mult=0.10, WD=0.0
  Group 2: 921,600 params, LR_mult=1.00, WD=0.1
Total optimizable parameters: 939,392
Using fused AdamW: True
Created a PretokDataset with rng seed 42
Starting training loop...
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 0: train loss 4.9223, val loss 4.9226
New best val loss: 4.9226
Saving checkpoint to out/ckpt.pt
wrote out/model.bin
0 | loss 4.9256 | lr 5.00e-

In [7]:
%%time
# inference
!cd ./modded-llama2.c && ./run out/model.bin -z data/tok128.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time vets mery mily. Thely locm toy toches nise on. One bide. They and goated poft a bigh her enends, the ganke nomy, the mommy. She mont hered, " jon a balle pind as on hery, Lit was toe doy sofly, the was movettiet pot play was a hat he hars and mas. " He her lime in the exce. Shent the deows it to borar. Bad wit er
achieved tok/s: 822.580645
CPU times: user 7.23 ms, sys: 1.16 ms, total: 8.39 ms
Wall time: 408 ms
